In [186]:
import pandas as pd
from datetime import datetime
import re

In [187]:
def open_file(filename):
    try:
        f = open(filename, "r")
        text = f.read()
    except:
        f = open(filename, "r",encoding='latin-1')
        text = f.read()
        # encoding='latin-1'
    lines = text.split('\n')
    return lines


def clean_name(name: str) -> str:
    char_add_ons = ['V.O.', "'S COM VOICE",
                    'Cont\'d', 'O.S.', "'S", "VOICE", "INTERCOM"]
    for txt in char_add_ons:
        if txt in name:
            name = name.replace(txt, '')
    return re.sub("[\(\[].*?[\)\]]", "", name.strip())


def get_title(lines):
    series_ind = 0
    for i in range(len(lines)):
        if "STAR TREK: THE NEXT GENERATION" in lines[i] or 'STAR TREK: DEEP SPACE NINE' in lines[i]:
            series_ind = i
            break
    return str(lines[series_ind+2]).strip().replace('"','')


def get_date(lines):
    draft_ind = 0
    for i in range(len(lines)):
        if "FINAL DRAFT" in lines[i]:
            series_ind = i
            break
    try:
        date = str(lines[series_ind+2])
        date = date.strip()
        print(date)
        final = datetime.strptime(date, '%B %d, %Y')
    except:
        print(lines[series_ind+2])
        final = "AHHH"
    return final



In [188]:
def get_quotes(lines):
#    characters = [x for x in lines if '\t\t\t\t\t' in x]
    char_index = [i for i in range(len(lines)) if '\t\t\t\t\t' in lines[i]]
    scenes_index = [j for j in range(len(lines)) if 'INT.' in lines[j] or 'EXT.' in lines[j]]

    data = []

    for i in char_index:
        name = lines[i].replace('\t\t\t\t\t', '')
        if "FADE OUT" in name:
            continue

        j = i+1
        next_line = lines[j]

        quote = ""

        while next_line != '':
            next_line = next_line.replace('\t', ' ')
            quote += re.sub("[\(\[].*?[\)\]]", "", next_line)
            j = j+1
            next_line = lines[j]

        if quote!="":
            print(quote,lines[j],lines[j-1])
            print(j)
            print(scenes_index)
            scene = lines[max([x for x in scenes_index if x < i])]

            data.append([name, quote,scene])

    test_df = pd.DataFrame(data, columns=['character', 'quote','scene'])
    return (test_df)

def clean_location(loc:str):
    views = ['INT.','EXT.']
    output = loc
    for v in views:
        if v in output:
            output = output[output.index(v)+5:]
    if '(' in output:
        output = output[:output.index('(')-1]
    elif '-' in output:
        output = output[:output.index('-')-1]
    return(output)

def get_view(loc:str):
    output = None
    views = ['INT.','EXT.']
    for v in views:
        if v in loc:
            output=v
    return(output)


def create_df(filepath: str) -> pd.DataFrame:
    lines = open_file(filepath)
    test_df = get_quotes(lines)
    test_df['location']=test_df['scene'].apply(clean_location)
    test_df['view']=test_df['scene'].apply(get_view)
    test_df['character'] = test_df['character'].apply(clean_name)
    test_df['episode'] = get_title(lines)
    test_df['date']=get_date(lines)
    return test_df


In [299]:
full_data=pd.read_csv('complete_data.csv',index_col=0)
date_issues = full_data[ full_data.date=='AHHH'].file.unique()
print(date_issues)

[]


In [303]:
[x for x in full_data.location.unique() if '-' in x]

['RUNABOUT\t49-',
 "QUARK'S -",
 "SHAKAAR'S FARM - DAY",
 'PROCESSING CENTER - NIGHT #3',
 "CHRIS BRYNNER'S OFFICE - NIGHT #3",
 'SPACE - THE DEFIANT',
 'DEFIANT - TRANSPORTER ROOM',
 'PROCESSING CENTER - NIGHT',
 'SAN FRANCISCO STREET - NIGHT',
 'PROCESSING CENTER - DAY #4',
 'SPACE - DEEP SPACE NINE',
 "COMMANDER'S OFFICE - CLOSE ON MONITOR",
 'HOLDING AREA - MOMENTS LATER',
 'A TAVERN - DAY',
 'TAVERN - NIGHT',
 'TAVERN - DAY',
 'DEFIANT - BRIDGE',
 'CARDASSIA - DAY',
 'BARRACKS - NIGHT',
 'DEFIANT - MEDICAL BAY',
 'SHAPE-SHIFTER SEA',
 'ATOLL - DAY',
 'WARDROOM - ON MONITOR',
 'ROMULAN QUARTERS - CONTINUOUS',
 'DEFIANT - JEFFERIES TUBE',
 'SPACE - THE DEFIANT - STOCK',
 'DEFIANT - ENGINE ROOM',
 'DEFIANT - CORRIDOR',
 'DEFIANT - ANOTHER JEFFERIES TUBE',
 'SPACE - DEEP SPACE NINE - STOCK',
 'RAIDER - TRANSPORTER AREA',
 'AIRLOCK CORRIDOR - CONTINUOUS',
 "DUKAT'S BIRD OF PREY - BRIDGE",
 "TY'GOKOR COMMAND CENTER - CORRIDOR",
 'DEEP SPACE NINE - SURGERY',
 'DOMINION INTERNMENT CENTER 

In [221]:
t='''STAR TREK: "Lower Decks" - 11/17/93 - CAST '''
re.search(r'\d{2}/\d{2}/\d{2}', t).group()

'11/17/93'

In [2]:
from os import listdir
from os.path import isfile, join
import pandas as pd

folders = {'scripts_ds9/':'Deep Space Nine','scripts_tng/':'The Next Generation'}
# get the files for each folder/series
file_dict = {}
for series in folders.keys():
    onlyfiles = [f for f in listdir(series) if isfile(join(series, f))]
    temp = {f:series for f in onlyfiles}
    # combine dict
    file_dict={**file_dict,**temp}

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [9]:
print(len(file_dict.keys()))

full_data=pd.read_csv('complete_data.csv',index_col=0)
full_data.shape

349


(144211, 9)